# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
import holoviews as hv
hv.extension('bokeh')
from bokeh.plotting import show

# Import API key
from api_keys import geoapify_key

c:\Users\charu\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("__pycache__/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,7.02,63,0,3.09,US,2024-02-12
1,1,blackmans bay,-43.0167,147.3167,12.07,93,88,1.21,AU,2024-02-12
2,2,det udom,14.9060,105.0784,18.90,78,86,1.64,TH,2024-02-12
3,3,waitangi,-43.9535,-176.5597,14.90,84,24,4.47,NZ,2024-02-12
4,4,oliveira dos brejinhos,-12.3169,-42.8961,29.77,45,100,4.28,BR,2024-02-12


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display
from IPython.display import display


map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    
)

# Display the map
hv.save(map_plot_1, 'map_plot.html')
show(hv.render(map_plot_1))



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.loc[(city_data_df["Max Temp"] < 22)&
                                   (city_data_df["Max Temp"] > 10)&
                                   (city_data_df["Humidity"] < 40)
                                   ]

print(len(clean_city_data))
# Drop any rows with null values
clean_cities = clean_city_data.dropna()

# Display sample data
clean_cities.head()


20


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,ontario,34.0633,-117.6509,18.62,36,20,2.57,US,2024-02-12
114,114,kutloanong,-27.8333,26.7500,21.95,36,1,3.12,ZA,2024-02-12
157,157,tura,25.5198,90.2201,14.28,28,25,1.25,IN,2024-02-12
167,167,bhilwara,25.3500,74.6333,14.84,39,39,2.15,IN,2024-02-12
184,184,quellon,-43.1167,-73.6167,19.28,32,9,3.43,CL,2024-02-12


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
92,ontario,US,34.0633,-117.6509,36,
114,kutloanong,ZA,-27.8333,26.7500,36,
157,tura,IN,25.5198,90.2201,28,
167,bhilwara,IN,25.3500,74.6333,39,
184,quellon,CL,-43.1167,-73.6167,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = clean_cities.loc[index,"Lat"]
    longitude = clean_cities.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
ontario - nearest hotel: Travelodge by Wyndham Ontario
kutloanong - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
bhilwara - nearest hotel: No hotel found
quellon - nearest hotel: Aliwen
letpandan - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
ciudad victoria - nearest hotel: Central
chonchi - nearest hotel: Hotel Huildín
guerrero negro - nearest hotel: Plaza sal paraiso
najran - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
san julian - nearest hotel: No hotel found
saidpur - nearest hotel: Hotel Amzadia International হোটেল আমজাদিয়া ইন্টারন্যাশনাল
rio verde - nearest hotel: Hotel Vista Bonita
port hueneme - nearest hotel: Americas Best Value Inn Oxnard Port Hueneme
serdar - nearest hotel: Chynar Hotel
tejen - nearest hotel: Tejen Myhmanhanasy
recani - nearest hotel: Hotel Merzouga
ghat - nearest hotel: فندق تاسيلي


,City,Country,Lat,Lng,Humidity,Hotel Name
92,ontario,US,34.0633,-117.6509,36,Travelodge by Wyndham Ontario
114,kutloanong,ZA,-27.8333,26.7500,36,No hotel found
157,tura,IN,25.5198,90.2201,28,No hotel found
167,bhilwara,IN,25.3500,74.6333,39,No hotel found
184,quellon,CL,-43.1167,-73.6167,32,Aliwen


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot


# Display the map
# YOUR CODE HERE

In [13]:
%%capture --no-display

map_plot_2 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1.5,
    color = "City",
    alpha = 0.6,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"],
    frame_width = 700,
    frame_height = 500,
    
)
hv.save(map_plot_2, 'map_plot.html')

show(hv.render(map_plot_2))
# display(map_plot_2)
# hv.save(map_plot_2, 'map_plot.html')
